# Datenvisualisierung für die Historische Forschung

Exploration verschiedener Strategien zur visuellen Analyse und Präsentation komplexer historischer Daten, u. a. Erstellung dynamischer Darstellungen wie Zeitstrahlen und Karten. Zudem wird die Möglichkeit der räumlichen Visualisierung thematischer Klassifikationen diskutiert (Pyplot, Wikibase Query Service).

## Übungen

Visualisierungen (Matplotlib / Pyplot) aus den aufbereiteten & angereicherten Daten (Tasks 1 & 2) sowie SPARQL-Abfragen gegen Wikidata zur Kontextualisierung. Nutzung von Property P10626 ("deckenmalerei.eu ID")

1. Einlesen von buildings_openrefine
2. Histogramm der Bau-/Datierungsjahre (Verwendung `building_dates` → Mittelwert pro Gebäude aus Range).
3. Berechnet die durchschnittliche Dauer zwischen `year_start` und `year_end` je Bundesland aus `addressState` und visualisiert sie als Balkendiagramm.

Machen diese Plots Sinn?


In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 1. Einlesen ohne Angabe von gewünschten Datentyp convertiert automatisch year_start und end in Integer
bd = pd.read_csv('buildings_openrefine.csv')
bd['year_mean'] = (bd.year_start + bd.year_end) / 2

# 2. Histogram mit den durschnittlichen Mitten der Datierungsranges
plt.figure(figsize=(8,4))
plt.hist(bd['year_mean'], bins=30, color='#446699')
plt.title('Verteilung Datierungsjahre (Mittel)')
plt.xlabel('Jahr')
plt.ylabel('Anzahl Segmente')
plt.tight_layout()
plt.show()

In [ ]:

# 3. Bauzeit
bd['bauzeit'] = bd['year_end'] - bd['year_start']
bd = bd[bd['bauzeit'] >= 0]

avg_bauzeit = bd.groupby('addressState')['bauzeit'].mean().sort_values(ascending=False)

plt.figure(figsize=(9, 5))
avg_bauzeit.plot(kind='bar', color='#7A9CC6')
plt.title('Durchschnittliche Bauzeit nach Bundesland')
plt.xlabel('Bundesland (addressState)')
plt.ylabel('Durchschnittliche Bauzeit (Jahre)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### 3. Nutzung von Property P10626 (deckenmalerei.eu ID)

Property-Link: <https://www.wikidata.org/wiki/Property:P10626>

Aufgaben:

1. Prüfe stichprobenartig für vorhandene `wikidata_id`s, ob die lokale UUID als P10626 Wert vorhanden ist (SPARQL Query → Validierung).

### 4. SPARQL Queries (Wikidata Query Service)

Karte mit allen Gebäuden mit deckenmalerei.eu Eigenschaft. Einfügen in den [Wikidata Query Service](https://query.wikidata.org).

```sparql
#title:location of items with property P10626 statements
#defaultView:Map{"hide":["?coordinates"]}
SELECT DISTINCT ?site ?siteLabel ?coordinates ?image ?value
WHERE 
{
  ?site wdt:P10626 ?value; wdt:P625 ?coordinates.
  OPTIONAL { ?site wdt:P18 ?image }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
```